# 1. 라이브러리 불러오기

In [1]:
import os
import sys
import re
import platform

import pandas as pd
import numpy as np
pd.set_option('max.columns', 1000)

import warnings
warnings.filterwarnings(action = 'ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'AppleGothic'

from sklearn.preprocessing import StandardScaler

# 2. 데이터 불러오기

## 2.1 서울시 주민등록인구(동별) 통계 데이터

In [2]:
# 서울시 주민등록인구 데이터 불러오기
person = pd.read_csv('./data/서울시 주민등록인구 (동별) 통계.txt', sep = '\t')
person.columns = ['기간', '자치구', '행정동', '세대', '총합계', '전체 남자', '전체 여자', '한국인 합계', '한국인 남자', '한국인 여자', '외국인 합계', '외국인 남자', '외국인 여자', '세대당인구', '65세이상고령자']

# 데이터 일부 확인
display(person.head())

,기간,자치구,행정동,세대,총합계,전체 남자,전체 여자,한국인 합계,한국인 남자,한국인 여자,외국인 합계,외국인 남자,외국인 여자,세대당인구,65세이상고령자
0,2019,합계,합계,"4,327,605","10,010,983","4,877,725","5,133,258","9,729,107","4,744,059","4,985,048","281,876","133,666","148,210",2.25,"1,485,272"
1,2019,종로구,소계,"73,947","161,869","78,215","83,654","151,290","73,746","77,544","10,579","4,469","6,110",2.05,"27,519"
2,2019,종로구,사직동,"4,621","9,815","4,455","5,360","9,535","4,297","5,238",280,158,122,2.06,"1,778"
3,2019,종로구,삼청동,"1,396","2,997","1,439","1,558","2,758","1,326","1,432",239,113,126,1.98,664
4,2019,종로구,부암동,"4,298","10,551","5,045","5,506","10,100","4,826","5,274",451,219,232,2.35,"1,762"


In [3]:
# 서울시 주민등록인구 데이터에서 '합계', '소계'는 필요없으므로 제거
person = person[(person['행정동'] != '합계') & (person['행정동'] != '소계')].reset_index(drop = True)

# 분석에 사용할 '자치구', '행정동', '총합계', '외국인 합계', '65세이상고령자' 변수 추출
person = person[['자치구', '행정동', '총합계', '외국인 합계']]

# 데이터 일부 확인
display(person.head())

,자치구,행정동,총합계,외국인 합계
0,종로구,사직동,"9,815",280
1,종로구,삼청동,"2,997",239
2,종로구,부암동,"10,551",451
3,종로구,평창동,"18,816",157
4,종로구,무악동,"8,740",45


In [4]:
# 총합계, 외국인 합계, 65세이상고령자 변수 콤마(,) 제거 후, 수치형으로 변경
for column in ['총합계', '외국인 합계']:
    person[column] = person[column].apply(lambda x: x.replace(',', '')).astype('int')
    
# 변수명 변경
person.rename(columns = {'총합계' : '총인구', '외국인 합계' : '외국인'}, inplace = True)

# 데이터 일부 확인
print('서울시 주민등록인구 (동별) 통계 데이터 행과열 :', person.shape, '\n')
display(person.head())

서울시 주민등록인구 (동별) 통계 데이터 행과열 : (424, 4) 



,자치구,행정동,총인구,외국인
0,종로구,사직동,9815,280
1,종로구,삼청동,2997,239
2,종로구,부암동,10551,451
3,종로구,평창동,18816,157
4,종로구,무악동,8740,45


## 2.2 서울시 가구원수별 가구수 (동별) 통계 데이터

In [5]:
# 서울시 가구원수별 가수구 통계 데이터 불러오기
seoul_gagu = pd.read_csv('./data/서울시 가구원수별 가구수 (동별) 통계.txt', sep = '\t')

# 데이터 일부 확인
display(seoul_gagu.head())

,기간,자치구,동,계,1인가구,2인가구,3인가구,4인가구,5인가구,6인가구,7인이상 가구
0,2020,합계,합계,"3,982,290","1,390,701","1,033,901","792,690","602,791","130,122","25,770","6,315"
1,2020,종로구,소계,"63,414","25,983","16,281","10,882","7,679","2,027",440,122
2,2020,종로구,사직동,"3,839","1,487","1,073",701,432,112,28,6
3,2020,종로구,삼청동,"1,031",380,295,189,112,39,11,5
4,2020,종로구,부암동,"3,773","1,129","1,089",772,552,176,39,16


In [6]:
# 동 변수에서 '합계', '소계' 제거
seoul_gagu = seoul_gagu[(seoul_gagu['동'] != '합계') & (seoul_gagu['동'] != '소계')]

# 사용할 변수 지정 후, 변수명 변경
seoul_gagu = seoul_gagu[['자치구', '동', '1인가구']]
seoul_gagu.rename(columns = {'동' : '행정동'}, inplace = True)

# 1인가구 변수가 문자형이므로 숫자형으로 변환
seoul_gagu['1인가구'] = seoul_gagu['1인가구'].apply(lambda x: x.replace(',', '')).astype('int')

# 데이터 일부 확인
display(seoul_gagu.head())

,자치구,행정동,1인가구
2,종로구,사직동,1487
3,종로구,삼청동,380
4,종로구,부암동,1129
5,종로구,평창동,1302
6,종로구,무악동,494


In [7]:
# '항동' 관측치만 따로 추출
hang_dong = seoul_gagu[seoul_gagu['행정동'] == '항동']

# 서울시 가구원수별 가구수 데이터에서 '항동' 제거 후, '오류2동' 관측치에 합해주기
seoul_gagu = seoul_gagu[seoul_gagu['행정동'] != '항동'].reset_index(drop = True)
seoul_gagu.loc[seoul_gagu['행정동'] == '오류2동', '1인가구'] += int(hang_dong['1인가구'])

# 데이터 일부 확인
print('서울시 가구원수별 가구수 통계 데이터 행과열 :', seoul_gagu.shape, '\n')
display(seoul_gagu.head())

서울시 가구원수별 가구수 통계 데이터 행과열 : (424, 3) 



,자치구,행정동,1인가구
0,종로구,사직동,1487
1,종로구,삼청동,380
2,종로구,부암동,1129
3,종로구,평창동,1302
4,종로구,무악동,494


## 2.3 서울시 주민등록인구 (연령별/동별) 통계

In [8]:
# 서울시 고령자수 데이터 불러오기
seoul_old = pd.read_csv('./data/pop_k_age_old.csv', encoding = 'CP949')
seoul_old.rename(columns = {'X65.69세' : '65 ~ 69세', 'X70.74세' : '70 ~ 74세', 'X75.79세' : '75 ~ 79세', 'X80.84세' : '80 ~ 84세', 
                            'X85.89세' : '85 ~ 89세', 'X90.94세' : '90 ~ 94세', 'X95.99세' : '95 ~ 99세', 'X100세.이상' : '100세 이상'}, inplace = True)

# 데이터 일부 확인
display(seoul_old.head())

,자치구,행정동,65 ~ 69세,70 ~ 74세,75 ~ 79세,80 ~ 84세,85 ~ 89세,90 ~ 94세,95 ~ 99세,100세 이상
0,종로구,사직동,446,410,374,287,142,68,21,19
1,종로구,삼청동,166,156,140,89,56,21,7,8
2,종로구,부암동,488,451,342,264,128,51,22,10
3,종로구,평창동,917,776,655,458,268,113,39,24
4,종로구,무악동,418,338,303,211,78,38,12,15


In [9]:
# 노인인구 변수 생성
seoul_old['노인인구'] = seoul_old.sum(axis = 1)

# 사용할 변수만 추출
seoul_old = seoul_old[['자치구', '행정동', '노인인구']]

# 데이터 일부 확인
print('서울시 노인인구 데이터 :', seoul_old.shape, '\n')
display(seoul_old.head())

서울시 노인인구 데이터 : (424, 3) 



,자치구,행정동,노인인구
0,종로구,사직동,1767
1,종로구,삼청동,643
2,종로구,부암동,1756
3,종로구,평창동,3250
4,종로구,무악동,1413


## 2.4 서울시 가구형태별 가구 및 가구원 (동별) 통계

In [10]:
# 서울시 가구형태별 가구 및 가구원 (동별) 통계 데이터 불러오기
seoul_gagu_type = pd.read_csv('./data/서울시 가구형태별 가구 및 가구원 (동별) 통계.txt', sep = '\t')
seoul_gagu_type.columns = ['기간', '자치구', '동', '가구수', '계', '남자', '여자', '일반가구 가구수', '일반가구 계', 
                           '일반가구 남자', '일반가구 여자', '집단비혈연 가구수', '집단비혈연 계', '집단비혈연 남자', 
                           '집단비혈연 여자', '집단가구 가구수', '집단가구 계', '집단가구 남자', '집단가구 여자', 
                           '외국인가구 가구수', '외국인가구 계', '외국인가구 남자', '외국인가구 여자']

# 데이터 일부 확인
display(seoul_gagu_type.head())

,기간,자치구,동,가구수,계,남자,여자,일반가구 가구수,일반가구 계,일반가구 남자,일반가구 여자,집단비혈연 가구수,집단비혈연 계,집단비혈연 남자,집단비혈연 여자,집단가구 가구수,집단가구 계,집단가구 남자,집단가구 여자,외국인가구 가구수,외국인가구 계,외국인가구 남자,외국인가구 여자
0,2020,합계,합계,"4,126,524","9,586,195","4,647,733","4,938,462","3,982,290","9,098,805","4,376,850","4,721,955","1,479","13,447","7,459","5,988",785,"79,188","33,026","46,162","141,970","264,013","129,752","134,261"
1,2020,종로구,소계,"68,069","151,291","73,062","78,229","63,414","135,582","65,104","70,478",68,585,399,186,47,"3,742","1,799","1,943","4,540","7,573","3,414","4,159"
2,2020,종로구,사직동,"3,969","8,705","3,959","4,746","3,839","8,234","3,697","4,537",X,37,16,21,X,92,46,46,123,197,110,87
3,2020,종로구,삼청동,"1,065","2,433","1,184","1,249","1,031","2,287","1,074","1,213",X,X,X,X,X,X,X,X,34,53,33,20
4,2020,종로구,부암동,"3,901","9,666","4,622","5,044","3,773","9,061","4,308","4,753",X,28,12,16,6,193,54,139,119,169,98,71


In [11]:
# 일반가구 데이터만 저장 후, 변수명 변경
seoul_gagu_type = seoul_gagu_type[['자치구', '동', '일반가구 가구수']]
seoul_gagu_type.columns = ['자치구', '행정동', '일반가구']

# 데이터 일부 확인
display(seoul_gagu_type.head())

,자치구,행정동,일반가구
0,합계,합계,"3,982,290"
1,종로구,소계,"63,414"
2,종로구,사직동,"3,839"
3,종로구,삼청동,"1,031"
4,종로구,부암동,"3,773"


In [12]:
# 행정동 변수에서 '합계', '소계' 제거
seoul_gagu_type = seoul_gagu_type[~seoul_gagu_type['행정동'].isin(['합계', '소계'])]

# 일반가구 변수가 문자형이므로 숫자형으로 변경
seoul_gagu_type['일반가구'] = seoul_gagu_type['일반가구'].apply(lambda x: x.replace(',', '')).astype('int')

display(seoul_gagu_type.head())

,자치구,행정동,일반가구
2,종로구,사직동,3839
3,종로구,삼청동,1031
4,종로구,부암동,3773
5,종로구,평창동,6265
6,종로구,무악동,2883


In [13]:
# 행정동 변수에서 '항동' 데이터 추출
hang_dong = seoul_gagu_type[seoul_gagu_type['행정동'] == '항동']

# 서울시 가구형태별 데이터에서 '항동' 제거 후, '오류2동' 관측치에 합해주기
seoul_gagu_type = seoul_gagu_type[seoul_gagu_type['행정동'] != '항동'].reset_index(drop = True)
seoul_gagu_type.loc[seoul_gagu_type['행정동'] == '오류2동', '일반가구'] += int(hang_dong['일반가구'])

# 데이터 일부 확인
print('서울시 가구형태별 가구 및 가구원 통계 데이터 :', seoul_gagu_type.shape, '\n')
display(seoul_gagu_type.head())

서울시 가구형태별 가구 및 가구원 통계 데이터 : (424, 3) 



,자치구,행정동,일반가구
0,종로구,사직동,3839
1,종로구,삼청동,1031
2,종로구,부암동,3773
3,종로구,평창동,6265
4,종로구,무악동,2883


# 3. 데이터 병합

In [14]:
# 주민등록인구, 가구원수별 데이터 JOIN
merge_data = pd.merge(person, seoul_gagu, on = ['자치구', '행정동'])

# 고령자수 데이터 JOIN
merge_data = pd.merge(merge_data, seoul_old, on = ['자치구', '행정동'])

# 일반가구 데이터 JOIN
merge_data = pd.merge(merge_data, seoul_gagu_type, on = ['자치구', '행정동'])

# 데이터 병합을 위해 '종로5·6가동' 바꿔주기
merge_data.loc[merge_data['행정동'] == '종로5·6가동', '행정동'] = '종로5.6가동'

# 데이터 일부 확인
print('서울시 행정동별 병합 데이터 :', merge_data.shape, '\n')
display(merge_data.head())

서울시 행정동별 병합 데이터 : (424, 7) 



,자치구,행정동,총인구,외국인,1인가구,노인인구,일반가구
0,종로구,사직동,9815,280,1487,1767,3839
1,종로구,삼청동,2997,239,380,643,1031
2,종로구,부암동,10551,451,1129,1756,3773
3,종로구,평창동,18816,157,1302,3250,6265
4,종로구,무악동,8740,45,494,1413,2883


In [15]:
# 이상치가 존재하면 상한, 하한값으로 대체해주는 함수 생성
def outlier_transformer(raw_data = None, column = None, weight = None):
    
    # 데이터 보존을 위해 데이터 복사
    data = raw_data.copy()
    
    # 1사분위수, 3사분위수 구하기
    quan_25 = np.quantile(data[column], 0.25)
    quan_75 = np.quantile(data[column], 0.75)
    
    IQR = quan_75 - quan_25
    IQR_weight = IQR * weight
    
    lowest = quan_25 - IQR_weight
    highest = quan_75 + IQR_weight
    
    # 이상치가 존재하는 경우 상한값, 하한값으로 변경
    data[column] = data[column].apply(lambda x: quan_75 if x >= highest else x)
    data[column] = data[column].apply(lambda x: quan_25 if x <= lowest else x)
    
    return data[column]

In [16]:
# 이상치 관측치를 변환
merge_data['총인구'] = outlier_transformer(raw_data = merge_data, column = '총인구', weight = 1.5)
merge_data['외국인'] = outlier_transformer(raw_data = merge_data, column = '외국인', weight = 1.5)
merge_data['1인가구'] = outlier_transformer(raw_data = merge_data, column = '1인가구', weight = 1.5)
merge_data['노인인구'] = outlier_transformer(raw_data = merge_data, column = '노인인구', weight = 1.5)
merge_data['일반가구'] = outlier_transformer(raw_data = merge_data, column = '일반가구', weight = 1.5)

In [17]:
# 변수 Scaling
standard = StandardScaler()
merge_data[['총인구', '외국인', '1인가구', '노인인구', '일반가구']] = standard.fit_transform(merge_data[['총인구', '외국인', '1인가구', '노인인구', '일반가구']])

# 주성분 분석을 통해 도출된 각 변수들의 적재량
# 대체_1인가구 : 0.46779226
# 대체_일반가구 : 0.48278117
# 대체_총인구 : 0.47773681 
# 대체_외국인 : 0.34242779 
# 대체_노인인구 : 0.45011497

# 병합된 데이터에서 인구지수 변수 만들어주기
merge_data['인구지수'] = 0.46779226 * merge_data['1인가구'] + 0.48278117 * merge_data['일반가구'] +\
                       0.47773681 * merge_data['총인구'] + 0.34242779 * merge_data['외국인'] + 0.45011497 * merge_data['노인인구']

# 데이터 일부 확인
print('병합 데이터 행과열 :', merge_data.shape, '\n')
display(merge_data.head())

병합 데이터 행과열 : (424, 8) 



,자치구,행정동,총인구,외국인,1인가구,노인인구,일반가구,인구지수
0,종로구,사직동,-1.580757,-0.419844,-0.983947,-1.274415,-1.563356,-2.687627
1,종로구,삼청동,-2.383307,-0.567919,-1.679607,-2.130290,-2.384373,-4.228778
2,종로구,부암동,-1.494122,0.197733,-1.208921,-1.282791,-1.582653,-2.553090
3,종로구,평창동,-0.521244,-0.864067,-1.100204,-0.145177,-0.854029,-1.537221
4,종로구,무악동,-1.707296,-1.268562,-1.607968,-1.543970,-1.842876,-3.586893


- 서울시 행정동별로 **인구지수** 생성 후, 주거건물대지면적, 카페점포수 변수를 추가하기 위해 데이터를 저장해주도록 한다.

```python
merge_data.to_csv('저장할 경로/서울시_행정동별_인구지수.csv', encoding = 'CP949')
```